In [ ]:
import json
from datetime import datetime as dt

In [ ]:
import spacy
import numpy as np
from spacy.en import STOP_WORDS

In [ ]:
fmt = "%Y-%m-%d %H:%M"

In [ ]:
date, texts = zip(*[(dt.strptime(p["published"], fmt), p["text"]) for p in json.load(open("data/jdc-raw.en.json"))])

In [ ]:
np.save("data/jdc-date", np.array(date, dtype=np.datetime64))

In [ ]:
nlp = spacy.load("en")

In [ ]:
def transform_doc(doc):
    
    def is_ent(ent):
        tag = ent.root.ent_type_ or ent.root.pos_
        
        cond = not((tag == "DATE") or ent.root.is_space or ent.root.like_num or ent.root.is_punct)
        return cond
    
    tokens = []
    for ent in doc.noun_chunks:
        if len(ent) == 1 and ent.lower_ in list(STOP_WORDS) + ["cameroon"]:
            continue
        
        if len(ent) > 1 and ent[0].lower_ in STOP_WORDS:
            ent = ent[1:]
        tokens.append(ent)
    
    return [token.lower_ for token in tokens if is_ent(token)]

def transform_texts(texts, out_file):
    
    out = open(out_file, "w")
    for text in texts:
        doc = nlp(text)
        line = ", ".join(transform_doc(doc)) + "\n"
        out.write(line)
    
    out.close()

In [ ]:
raw = texts[0]
doc = nlp(raw)

tokens = transform_doc(doc)

In [ ]:
print(raw)

In [ ]:
print(" ".join(tokens))

In [ ]:
transform_texts(texts, "data/jdc-tokens.en.text")